In [1]:
import pandas as pd

In [2]:
class ProfessorDumpToPandas:
    def __init__(self, csv_path: str):
        self.csv_path = csv_path
        self.df = self.read_csv()
    
    def read_csv(self):
        return pd.read_csv(self.csv_path)


In [4]:
pdump = ProfessorDumpToPandas('/mnt/linuxlab/home/spresley1/Desktop/425Reset/Rommel-Center-Research/PythonCode/Utilities/RuntimeFiles/professor_department_dump.csv')
pdump_df = pdump.df

In [5]:
official_professor_names_and_dept: dict[str, str] = {}

for row in pdump_df.itertuples(index=False):
    key = f"{row.lastName}, {row.firstName}"
    if key not in official_professor_names_and_dept:
        official_professor_names_and_dept[key] = [row.departmentCode]
    else:
        official_professor_names_and_dept[key].append(row.departmentCode)

In [6]:
official_faculty_set: set[str] = set(official_professor_names_and_dept.keys())
print(official_faculty_set)

{'Diamonte-Mock, Christina M', 'Endicott, Seth Andrew', 'McCarty, Michael B', 'Crowley, Caitlyn Howard', 'Willey, Jeffrey Allan', 'Kelly, Clare Kathleen', "Daniels, Maria La'Trice", 'Eutsler, Madysen Marie', 'Owens-King, Allessia P', 'Lin, Wei-Ting', 'Campbell, Rita G', 'Good-Malloy, Nicholas', 'Collins, Khalilah V.', 'Benyish Jr, Joseph W', 'Dziwenka, Ronald J', 'Culotta, Jordan Jeffrey', 'Fagan, Kathleen H', 'Cashman, Burton Roger', 'Ehrhardt, Ursula M.', 'Money, Jessica Paige', 'Badros, Karen Kathryn', 'Shea, Michael P', 'Hesser, Phillip', 'Finch, Maida A', 'Nash, Naomi Ruth', 'Choi, Yoojin', 'Venso, Elichia A.', 'Knight, Margaret J.', 'White, Kylie M.', 'Clement, Grace A.', 'Stratton, Casey M', 'Massey, Kevin Dale', 'Chambers, Silvana S', 'Tardiff, Robert M.', 'Martino, Andrew P', 'Elburn, Sara J.', 'Perret, Sally A', 'Ortlip, Austin Tyler', 'Sessoms, Chalarra A', 'Kjeldsen, Joan Bromhall', 'Ross, Michael C', 'Young, Kevin', 'Imbrenda, Jon-Philip', 'Flores, Brian M', 'Morrison-Park

In [7]:
from My_Data_Classes import CategoryInfo
import pickle
def open_dict_pkl(file_path: str):
    with open(file_path, 'rb') as f:
        loaded_dict = pickle.load(f)
    return loaded_dict

In [8]:
cat_dict = open_dict_pkl('./category_dict.pkl')


In [9]:
unique_fac_set: set[str] = set()

for category, category_info in cat_dict.items():
    for faculty in category_info.faculty:
        unique_fac_set.add(faculty)
        
print(unique_fac_set)

{'Fowler, Garrett', 'Labb, Samantha A.', 'Wang, Shuangquan', 'Wille, Brendan', 'Morningred, Connor', 'Kulavuz-Onal, Derya', 'Warman, Stephanie', 'Ortlip, Austin T.', 'Roose, Jordan J.', 'Rexroth, Kayla S.', 'Townsend, Zachary', 'Lei, Shan', 'Walton, Kristen Post', 'McCartney, Jason', 'Boster, Charles R.', 'Nibblett, Kaitlyn', 'Howell, Julia', 'Van de Wide, Aurelie', 'Cha, Hoon S.', 'Mann, Hunter', 'Benner, Jeffrey', 'Norman, Brandon', 'Bushera, Hakeem', 'Kim, Koomi', 'Jermain, Madison', 'Bhatti, Khadija', 'Wu, Ying', 'Montgomery, Chandini B.', 'Talbot, John', 'Briand, Christopher H.', 'Choi, Yoojin', 'Bowler, Matthew M.', 'Peixoto, Sinelia', 'Treuth, Margarita S.', 'Whitehead, George, I', 'Pasirayi, Simba', 'Vicens, Belen', 'Joyner, Robert L., Jr.', 'Wulf, Isabel Quintana', 'Muller, Mark W.', 'French, Kara M.', 'Kim, Chan', 'Follmer, Kayla', 'Gonzalez, Aston', 'Leaver, Echo', 'Molina, Michael', 'Kolstoe, Sonja', 'Hunter, Kimberly L.', 'Osman, Suzanne L.', 'Patel, Shruti', 'Brown, Vonce

In [53]:
from fuzzywuzzy import fuzz, process
from itertools import combinations
import collections

name_groups = collections.defaultdict(list)
for name in unique_fac_set:
    if name == "":
        continue
    if ", " in name:
        last_name, remainder = name.split(", ", 1)
        first_initial = remainder[0] if remainder else ''
    else:
        # If there's no comma, split by the first space.
        parts = name.split(" ", 1)
        if len(parts) == 2:
            last_name, remainder = parts
            first_initial = remainder[0] if remainder else ''
        else:
            # Handle case where there's no space (single-word name)
            last_name = name
            first_initial = ''
    group_key = (last_name, first_initial)
    name_groups[group_key].append(name)

In [40]:
# Function to check similarity
count = 0  # Initialize count at the global scope
count2 = 0
def check_similarity(names):
    global count  # Declare count as global to modify the global variable inside this function
    pairs = combinations(names, 2)
    for item1, item2 in pairs:
        # Extract the first names and consider only up to the first space (if present)
        first_name1 = item1.split(", ")[1].split(" ")[0] if ", " in item1 else ''
        first_name2 = item2.split(", ")[1].split(" ")[0] if ", " in item2 else ''
        
        # Check if the first letters of the first names match and compare the lengths of the first names
        if first_name1[0] == first_name2[0] and len(first_name1) == len(first_name2):
            similarity = fuzz.ratio(item1, item2)
            print(f'Names "{item1}" and "{item2}" have a similarity of {similarity}%')
            count += 1
                
def check_similarity_3(names):
    global count2  # Declare count as global to modify the global variable inside this function
    pairs = combinations(names, 3)
    for item1, item2, item3 in pairs:
        # Extract the first names and consider only up to the first space (if present)
        first_name1 = item1.split(", ")[1].split(" ")[0] if ", " in item1 else ''
        first_name2 = item2.split(", ")[1].split(" ")[0] if ", " in item2 else ''
        first_name3 = item3.split(", ")[1].split(" ")[0] if ", " in item3 else ''

        
        # Check if the first letters of the first names match and compare the lengths of the first names
        if (first_name1[0] == first_name2[0] and len(first_name1) == len(first_name2) and first_name1[0] == first_name3[0] and len(first_name1) == len(first_name3)) or (first_name2[0] == first_name3[0] and len(first_name2) == len(first_name3)):
            similarity = fuzz.ratio(item1, item2)
            similarity2 = fuzz.ratio(item2, item3)
            print(f'Names "{item1}" and "{item2}" have a similarity of {similarity}%')
            print(f'Names "{item2}" and "{item3}" have a similarity of {similarity}%')
            count2 += 1
                
# Iterate over the groups and check similarity within each group
for group in name_groups.values():
    if len(group) > 1 and len(group) < 3:  # Only compare if there are at least 2 names in the group
        #print(f"{group}")
        check_similarity(group)
    elif len(group) > 2:
        print(f"\n{group}")
        check_similarity_3(group)
print(f"\nNumber of pairs that share the same last name, same first initial, and same first name length: {count}")
print(f"\nNumber of triplets that share the same last name, same first initial, and same first name length: {count2}")

Names "Ortlip, Austin T." and "Ortlip, Austin" have a similarity of 90%
Names "Boster, Charles R." and "Boster, Charles" have a similarity of 91%
Names "Whitehead, George, I" and "Whitehead, George I." have a similarity of 95%
Names "Follmer, Kayla" and "Follmer, Kayla B." have a similarity of 90%
Names "Green, Daniel" and "Green, Daniel C." have a similarity of 90%
Names "Nyland, Jennifer F." and "Nyland, Jennifer" have a similarity of 91%

['Naing, Yadanar T.', 'Naing, Yadanar Than', 'Naing, Yadanar']
Names "Naing, Yadanar T." and "Naing, Yadanar Than" have a similarity of 89%
Names "Naing, Yadanar Than" and "Naing, Yadanar" have a similarity of 89%
Names "Stutelberg, Erin B." and "Stutelberg, Erin" have a similarity of 91%
Names "Pellinger, Thomas K." and "Pellinger, Thomas" have a similarity of 92%
Names "Singh, Nitya P." and "Singh, Nitya" have a similarity of 89%
Names "Smith, Kenneth" and "Smith, Kenneth J." have a similarity of 90%
Names "Kotlowski, Dean J." and "Kotlowski, Dea

In [18]:
print(name_groups.keys())

dict_keys([('Fowler', 'G'), ('Labb', 'S'), ('Wang', 'S'), ('Wille', 'B'), ('Morningred', 'C'), ('Kulavuz-Onal', 'D'), ('Warman', 'S'), ('Ortlip', 'A'), ('Roose', 'J'), ('Rexroth', 'K'), ('Townsend', 'Z'), ('Lei', 'S'), ('Walton', 'K'), ('McCartney', 'J'), ('Boster', 'C'), ('Nibblett', 'K'), ('Howell', 'J'), ('Van de Wide', 'A'), ('Cha', 'H'), ('Mann', 'H'), ('Benner', 'J'), ('Norman', 'B'), ('Bushera', 'H'), ('Kim', 'K'), ('Jermain', 'M'), ('Bhatti', 'K'), ('Wu', 'Y'), ('Montgomery', 'C'), ('Talbot', 'J'), ('Briand', 'C'), ('Choi', 'Y'), ('Bowler', 'M'), ('Peixoto', 'S'), ('Treuth', 'M'), ('Whitehead', 'G'), ('Pasirayi', 'S'), ('Vicens', 'B'), ('Joyner', 'R'), ('Wulf', 'I'), ('Muller', 'M'), ('French', 'K'), ('Kim', 'C'), ('Follmer', 'K'), ('Gonzalez', 'A'), ('Leaver', 'E'), ('Molina', 'M'), ('Kolstoe', 'S'), ('Hunter', 'K'), ('Osman', 'S'), ('Patel', 'S'), ('Brown', 'V'), ('Williams', 'E'), ('Demond', 'W'), ('Jung', 'K'), ('Ali', 'B'), ('Green', 'D'), ('Maykrantz', 'S'), ('Insley', 'C

In [19]:
def is_name_a_group_key(name, name_groups):
    # Reformat name input so it's just the lastname and first initial in order to match the key format in name_groups
    if ", " in name:
        last_name, remainder = name.split(", ", 1)
        first_initial = remainder[0] if remainder else ''
    else:
        # If there is no comma split by the first space
        parts = name.split(" ", 1)
        if len(parts) == 2:
            last_name, remainder = parts
            first_initial = remainder[0] if remainder else ''
        else:
            # For the case a name is a single word
            last_name = name
            first_initial = ''

    # create a key that matches the key formats in name_groups
    group_key = (last_name, first_initial)
    
    # check if the key exists in name_groups
    if group_key in name_groups:
        return True, name_groups[group_key]
    else:
        return False, []

In [57]:
import re

unique_fac_set: set[str] = set()
pattern = r'^[A-Za-z]+,\s[A-Za-z]+\s[A-Z]\.$'


for category, category_info in cat_dict.items():
    for faculty in category_info.faculty:
        unique_fac_set.add(faculty)

refined_unique_fac_set: set[str] = set()
for name in unique_fac_set:
    a_group_key = is_name_a_group_key(name, name_groups)
    if a_group_key[0]:
        values = a_group_key[1]
        desired_value: str = ""
        for value in values:
            if re.match(pattern, value):
                desired_value = value
            else:
                desired_value = max(values, key=len)
            refined_unique_fac_set.add(desired_value)
            
# print(unique_fac_set)
# print(refined_unique_fac_set)
print(f"Num Elements in UniqueFacSet: {len(unique_fac_set)}")
print(f"Num Elements in RefinedFacSet: {len(refined_unique_fac_set)}")

new_set: set[str] = unique_fac_set - refined_unique_fac_set
# print(f"Num Elements in NewSet: {len(new_set)}")

new_set_list: list[str] = list(new_set)

last_names = {name.split(", ")[0] for name in new_set}

refined_unique_fac_list = list(refined_unique_fac_set)

print("\n")
best_matches: list[str] = []
for last_name in last_names:
    best_match, score = process.extractOne(last_name, refined_unique_fac_list)
    best_matches.append(best_match)

best_matches.sort()
new_set_list.sort()

for i in range(len(max(best_matches, new_set_list))):
    print(f"{best_matches[i]}: {new_set_list[i]}")



Num Elements in UniqueFacSet: 469
Num Elements in RefinedFacSet: 425


Allen, Kimberly: Allen, Kim
Billups, M. Judith: Billups, M. J.
Boster, Charles R.: Boster, Charles
Bressman, N. R.: Bressman, Noah
Briand, Christopher H.: Briand, C. H.
Caviglia-Harris, Jill L.: Caviglia-Harris, Jill
Chen, Mara: Chen, Xuan
Clark, Jessica Kennett: Clark, Jessica
DiBartolo, Mary C.: DiBartolo, Mary
Emerson, Jamie: Emerson, David
Emmert, Elizabeth A. B.: Emmert, Elizabeth
Follmer, D. Jake: Follmer, Kayla
Fox, James T.: Fox, James
Franzak, Judith K.: Franzak, Judith
Geleta, Samuel B.: Geleta, S. B.
Green, Daniel C.: Green, Daniel
Groth, Randall E.: Groth, Randall
Habay, Stephen A.: Habay, Stephen
Hamilton, Stuart E.: Hamilton, Stu
Hoffman, Richard C.: Hamilton, Stuart
Joyce, Ryan Patrick: Hoffman, Richard
Kim, Koomi: Joyce, Ryan
Kotlowski, Dean J.: Kim, Yun-Kyoung
Maykrantz, Sherry A.: Kotlowski, Dean
Mazzetti, Scott: Maykrantz, S. A.
Miller, Stephanie: Miller, Jenna
Mitchell, Kelsey M.: Mitchell, Kaitl

In [58]:
for item in refined_unique_fac_list:
    print(item)

Fowler, Garrett
Wang, Shuangquan
Labb, Samantha A.
Wille, Brendan
Morningred, Connor
Kulavuz-Onal, Derya
Warman, Stephanie
Ortlip, Austin T.
Roose, Jordan J.
Rexroth, Kayla S.
Townsend, Zachary
Lei, Shan
Walton, Kristen Post
McCartney, Jason
Boster, Charles R.
Nibblett, Kaitlyn
Howell, Julia
Van de Wide, Aurelie
Cha, Hoon S.
Mann, Hunter
Benner, Jeffrey
Norman, Brandon
Bushera, Hakeem
Kim, Koomi
Jermain, Madison
Bhatti, Khadija
Wu, Ying
Montgomery, Chandini B.
Talbot, John
Briand, Christopher H.
Choi, Yoojin
Bowler, Matthew M.
Treuth, Margarita S.
Peixoto, Sinelia
Whitehead, George, I
Vicens, Belen
Joyner, Robert L., Jr.
Wulf, Isabel Quintana
Muller, Mark W.
French, Kara M.
Kim, Chan
Gonzalez, Aston
Leaver, Echo
Molina, Michael
Kolstoe, Sonja
Hunter, Kimberly L.
Osman, Suzanne L.
Patel, Shruti
Brown, Voncelia
Williams, E. Eugene
Demond, Whitney
Jung, Kyoung-Rae
Ali, Bakr
Insley, Carlton R., III
Richardson, Gwynne
Daly, E. Susanne
Timmons, Miranda
Corfield, Jeremy R.
Albright, Savannah 